In [1]:
import pandas as pd

df = pd.read_csv('data/Divvy_Trips_2020_Q1.csv')
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,EACB19130B0CDA4A,docked_bike,2020-01-21 20:06:59,2020-01-21 20:14:30,Western Ave & Leland Ave,239,Clark St & Leland Ave,326.0,41.9665,-87.6884,41.9671,-87.6674,member
1,8FED874C809DC021,docked_bike,2020-01-30 14:22:39,2020-01-30 14:26:22,Clark St & Montrose Ave,234,Southport Ave & Irving Park Rd,318.0,41.9616,-87.6660,41.9542,-87.6644,member
2,789F3C21E472CA96,docked_bike,2020-01-09 19:29:26,2020-01-09 19:32:17,Broadway & Belmont Ave,296,Wilton Ave & Belmont Ave,117.0,41.9401,-87.6455,41.9402,-87.6530,member
3,C9A388DAC6ABF313,docked_bike,2020-01-06 16:17:07,2020-01-06 16:25:56,Clark St & Randolph St,51,Fairbanks Ct & Grand Ave,24.0,41.8846,-87.6319,41.8918,-87.6206,member
4,943BC3CBECCFD662,docked_bike,2020-01-30 08:37:16,2020-01-30 08:42:48,Clinton St & Lake St,66,Wells St & Hubbard St,212.0,41.8856,-87.6418,41.8899,-87.6343,member


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426887 entries, 0 to 426886
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             426887 non-null  object 
 1   rideable_type       426887 non-null  object 
 2   started_at          426887 non-null  object 
 3   ended_at            426887 non-null  object 
 4   start_station_name  426887 non-null  object 
 5   start_station_id    426887 non-null  int64  
 6   end_station_name    426886 non-null  object 
 7   end_station_id      426886 non-null  float64
 8   start_lat           426887 non-null  float64
 9   start_lng           426887 non-null  float64
 10  end_lat             426886 non-null  float64
 11  end_lng             426886 non-null  float64
 12  member_casual       426887 non-null  object 
dtypes: float64(5), int64(1), object(7)
memory usage: 42.3+ MB


First I will change the data types of the columns to the correct ones

In [5]:
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])

df['trip_duration'] = df['ended_at'] - df['started_at']
df['duration_seconds'] = df['trip_duration'].dt.total_seconds()
df.head(4)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,trip_duration,duration_seconds
0,EACB19130B0CDA4A,docked_bike,2020-01-21 20:06:59,2020-01-21 20:14:30,Western Ave & Leland Ave,239,Clark St & Leland Ave,326.0,41.9665,-87.6884,41.9671,-87.6674,member,0 days 00:07:31,451.0
1,8FED874C809DC021,docked_bike,2020-01-30 14:22:39,2020-01-30 14:26:22,Clark St & Montrose Ave,234,Southport Ave & Irving Park Rd,318.0,41.9616,-87.6660,41.9542,-87.6644,member,0 days 00:03:43,223.0
2,789F3C21E472CA96,docked_bike,2020-01-09 19:29:26,2020-01-09 19:32:17,Broadway & Belmont Ave,296,Wilton Ave & Belmont Ave,117.0,41.9401,-87.6455,41.9402,-87.6530,member,0 days 00:02:51,171.0
3,C9A388DAC6ABF313,docked_bike,2020-01-06 16:17:07,2020-01-06 16:25:56,Clark St & Randolph St,51,Fairbanks Ct & Grand Ave,24.0,41.8846,-87.6319,41.8918,-87.6206,member,0 days 00:08:49,529.0


Now  I will delete duplicated rows, look for nulls and outliers

In [7]:
df.isnull().sum()

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      1
end_station_id        1
start_lat             0
start_lng             0
end_lat               1
end_lng               1
member_casual         0
trip_duration         0
duration_seconds      0
dtype: int64

In [8]:
df.drop_duplicates(inplace=True)

In [9]:
df.dropna(inplace=True)

In [26]:
# I will consider as outliers the trips that last less than 1 minute and more than 24 hours
df = df[df['duration_seconds'] >= 60]
df = df[df['duration_seconds'] <= 60 * 60 * 24].sort_values('duration_seconds', ascending=False)

Now I will create new features in order to improve the future analysis of the data by starting dates.

In [ ]:
df['starting_month'] = df['started_at'].dt.month
df['starting_hour'] = df['started_at'].dt.hour
df['starting_date'] = df['started_at'].dt.day_name()


I delete some features that are not useful for the analysis

In [35]:
df.drop('rideable_type', axis=1, inplace=True)

In [37]:
#As we have created new time features, now we can delete the old ones
df.drop(['started_at', 'ended_at', 'trip_duration'], axis=1, inplace=True)

In [38]:
df

,ride_id,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration_seconds,starting_month,starting_hour,starting_date
261357,EB83072244DF5183,Clark St & Drummond Pl,220,Millennium Park,90.0,41.9312,-87.6443,41.8810,-87.6241,casual,86155.0,2,11,Wednesday
335550,1924EA914F6FC007,Federal St & Polk St,41,Field Blvd & South Water St,7.0,41.8721,-87.6295,41.8863,-87.6175,casual,85990.0,3,18,Friday
205310,EB48B0C5B8A7E284,Dearborn St & Monroe St,49,Michigan Ave & 14th St,168.0,41.8813,-87.6295,41.8641,-87.6237,member,85928.0,2,17,Friday
120547,D77022E99184DCFC,Michigan Ave & Pearson St,25,Wabash Ave & Grand Ave,199.0,41.8977,-87.6235,41.8915,-87.6268,casual,85684.0,1,10,Wednesday
304642,0EE1ADD10A6B94A9,Racine Ave & Fullerton Ave,87,Clifton Ave & Armitage Ave,223.0,41.9256,-87.6584,41.9182,-87.6569,member,85302.0,3,7,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200317,78A1780A232B16F8,Canal St & Adams St,192,Canal St & Adams St,192.0,41.8793,-87.6399,41.8793,-87.6399,member,60.0,2,7,Thursday
77228,794157223DE8B9E5,Franklin St & Lake St,164,Franklin St & Lake St,164.0,41.8858,-87.6355,41.8858,-87.6355,member,60.0,1,17,Wednesday
176358,FD491D9B86420652,Clinton St & Washington Blvd,91,Canal St & Madison St,174.0,41.8834,-87.6412,41.8821,-87.6398,member,60.0,2,18,Wednesday
289880,9E7EB9668B2D6745,Clark St & Wrightwood Ave,340,Clark St & Drummond Pl,220.0,41.9295,-87.6431,41.9312,-87.6443,member,60.0,3,20,Tuesday


In [41]:
df.to_csv('data/cleanned_trips_2020_Q1.csv', index=False)

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 418926 entries, 261357 to 50920
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             418926 non-null  object 
 1   start_station_name  418926 non-null  object 
 2   start_station_id    418926 non-null  int64  
 3   end_station_name    418926 non-null  object 
 4   end_station_id      418926 non-null  float64
 5   start_lat           418926 non-null  float64
 6   start_lng           418926 non-null  float64
 7   end_lat             418926 non-null  float64
 8   end_lng             418926 non-null  float64
 9   member_casual       418926 non-null  object 
 10  duration_seconds    418926 non-null  float64
 11  starting_month      418926 non-null  int32  
 12  starting_hour       418926 non-null  int32  
 13  starting_date       418926 non-null  object 
dtypes: float64(6), int32(2), int64(1), object(5)
memory usage: 44.7+ MB
